## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

## Лабораторная работа 3

In [ ]:
!pip install line_profiler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.9/661.9 kB 16.4 MB/s eta 0:00:00


1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [3]:
import pandas as pd
import time

recipes = pd.read_csv("recipes_sample.csv", sep=",", parse_dates=['submitted'])
recipes = recipes.set_index('id')

reviews = pd.read_csv("reviews_sample.csv", sep=",",parse_dates=['date'])
reviews.rename(columns={'Unnamed: 0': 'id'}, inplace=True)
reviews = reviews.set_index('id')

reviews['date'] = pd.to_datetime(reviews['date'])
reviews['rating'] = reviews['rating'].astype(float)

In [4]:
def rating_A(reviews):
    start_time = time.time()
    total = 0
    count = 0
    for index, row in reviews.iterrows():
        if row['date'].year == 2010:
            total += row['rating']
            count += 1
    full_time = time.time() - start_time
    return "Cр.знач:",total/count,"Время:",full_time
rating_A(reviews)


('Cр.знач:', 4.4544402182900615, 'Время:', 6.088072299957275)

In [5]:
def rating_B(reviews):
    start_time = time.time()
    reviews_2010 = reviews[reviews['date'].dt.year == 2010]
    total = 0
    count = 0
    for index, row in reviews_2010.iterrows():
        total += row['rating']
        count += 1
    full_time = time.time() - start_time
    return "Cр.знач:",total/count,"Время:",full_time
rating_B(reviews)

('Cр.знач:', 4.4544402182900615, 'Время:', 0.8286757469177246)

In [6]:
def rating_C(reviews):
    start_time = time.time()
    reviews_2010 = reviews[reviews['date'].dt.year == 2010]
    answer = reviews_2010['rating'].mean()
    full_time = time.time() - start_time
    return "Cр.знач:",answer,"Время:",full_time
rating_C(reviews)

('Cр.знач:', 4.4544402182900615, 'Время:', 0.03578782081604004)

2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

In [16]:
!pip install line_profiler
%load_ext line_profiler
import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [14]:
%lprun -f rating_A rating_A(reviews)

In [17]:
def rating_4():
    start_time = time.time()
    data = np.matrix((reviews["rating"], reviews["date"].dt.year), dtype=int)
    mask = data[1,] == 2010
    faster_values = np.where(mask, data, 0)
    answer = faster_values[0,].sum() / mask.sum()
    full_time = time.time() - start_time
    return "Cр.знач:",answer,"Время:",full_time
rating_4()

('Cр.знач:', 4.4544402182900615, 'Время:', 0.026181459426879883)

3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [10]:
from line_profiler import LineProfiler


def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

data = {
    'recipe_id': [1, 2, 3],
    'review': ['This is a great recipe', 'I loved it', 'Awesome dish']
}
df = pd.DataFrame(data)

profiler = LineProfiler()
profiler.add_function(get_word_reviews_count)

profiler.enable_by_count()
word_reviews_count = get_word_reviews_count(df)
profiler.disable_by_count()

profiler.print_stats()

Timer unit: 1e-09 s

Total time: 0.0121661 s
File: <ipython-input-10-d65c752fe88d>
Function: get_word_reviews_count at line 4

Line #      Hits         Time  Per Hit   % Time  Line Contents
     4                                           def get_word_reviews_count(df):
     5         1       2806.0   2806.0      0.0      word_reviews = {}
     6         3    7567110.0 2522370.0     62.2      for _, row in df.dropna(subset=['review']).iterrows():
     7         3     166754.0  55584.7      1.4          recipe_id, review = row['recipe_id'], row['review']
     8         3       7349.0   2449.7      0.1          words = review.split(' ')
     9        10       6395.0    639.5      0.1          for word in words:
    10        10       6301.0    630.1      0.1              if word not in word_reviews:
    11        10       8226.0    822.6      0.1                  word_reviews[word] = []
    12        10      11693.0   1169.3      0.1              word_reviews[word].append(recipe_id)
    

In [11]:
from collections import defaultdict

def get_word_reviews_count(df):
    word_reviews = defaultdict(list)
    word_reviews_count = {}
    
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in set(words):
            word_reviews[word].append(recipe_id)
    
    for word, reviews in word_reviews.items():
        word_reviews_count[word] = len(reviews)
    
    return word_reviews_count


data = {
    'recipe_id': [1, 2, 3],
    'review': ['This is a great recipe', 'I loved it', 'Awesome dish']
}
df = pd.DataFrame(data)

profiler = LineProfiler()
profiler.add_function(get_word_reviews_count)

profiler.enable_by_count()
word_reviews_count = get_word_reviews_count(df)
profiler.disable_by_count()

profiler.print_stats()

Timer unit: 1e-09 s

Total time: 0.00897181 s
File: <ipython-input-11-75545a191850>
Function: get_word_reviews_count at line 4

Line #      Hits         Time  Per Hit   % Time  Line Contents
     4                                           def get_word_reviews_count(df):
     5         1       4008.0   4008.0      0.0      word_reviews = defaultdict(list)
     6         1        363.0    363.0      0.0      word_reviews_count = {}
     7                                               
     8         3    8808024.0 2936008.0     98.2      for _, row in df.dropna(subset=['review']).iterrows():
     9         3     112219.0  37406.3      1.3          recipe_id, review = row['recipe_id'], row['review']
    10         3      12060.0   4020.0      0.1          words = review.split(' ')
    11        10      10023.0   1002.3      0.1          for word in set(words):
    12        10      12926.0   1292.6      0.1              word_reviews[word].append(recipe_id)
    13                         

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [ ]:
import pandas as pd

def MAPE(df: pd.DataFrame) -> float:
    # Удаляем строки с нулевым рейтингом
    df = df[df['rating'] != 0]
    
    # Группируем данные по рецепту
    grouped_data = df.groupby('recipe_id')
    
    # Вычисляем средний рейтинг для каждого рецепта
    mean_ratings = grouped_data['rating'].mean()
    
    # Вычисляем сумму абсолютных процентных отклонений для каждого рецепта
    mape_sum = 0
    for recipe_id, rating in mean_ratings.iteritems():
        recipe_data = df[df['recipe_id'] == recipe_id]
        actual_rating = recipe_data['rating'].values
        mape_sum += abs(actual_rating - rating) / rating
    
    # Вычисляем среднее абсолютное процентное отклонение для всех рецептов
    result = mape_sum.sum() / len(mean_ratings)
    
    return result

In [ ]:
import timeit

def mape_v1(reviews):
    result = 0
    count = 0
    for recipe_id, recipe_group in reviews.groupby('recipe_id'):
        mean_rating = recipe_group['rating'].mean()
        for _, row in recipe_group.iterrows():
            if row['rating'] != 0:
                result += abs(row['rating'] - mean_rating) / mean_rating
                count += 1
    return result / count if count > 0 else 0

print(timeit.timeit(lambda: mape_v1(reviews), number=10, globals=globals())) # Для первой реализации

197.17042308500004


In [ ]:
import pandas as pd
from numba import jit

@jit(nopython=True)
def mape_numba(actual_rating, rating):
    mape_sum = 0
    for i in range(len(actual_rating)):
        mape_sum += abs(actual_rating[i] - rating) / rating
    return mape_sum

def MAPE(df: pd.DataFrame) -> float:
    # Удаляем строки с нулевым рейтингом
    df = df[df['rating'] != 0]
    
    # Группируем данные по рецепту
    grouped_data = df.groupby('recipe_id')
    
    # Вычисляем средний рейтинг для каждого рецепта
    mean_ratings = grouped_data['rating'].mean()
    
    # Вычисляем сумму абсолютных процентных отклонений для каждого рецепта
    mape_sum = 0
    for recipe_id, rating in mean_ratings.iteritems():
        recipe_data = df[df['recipe_id'] == recipe_id]
        actual_rating = recipe_data['rating'].values
        mape_sum += mape_numba(actual_rating, rating)
    
    # Вычисляем среднее абсолютное процентное отклонение для всех рецептов
    result = mape_sum.sum() / len(mean_ratings)
    
    return result

In [ ]:
import numpy as np
import timeit

def mape_v3(reviews):
    reviews = reviews[reviews['rating'] != 0]
    mean_ratings = reviews.groupby('recipe_id')['rating'].transform('mean')
    return np.mean(np.abs(reviews['rating'] - mean_ratings) / mean_ratings)

print(timeit.timeit(lambda: mape_v3(reviews), number=10, globals=globals())) # Для третьей реализации

0.35992541599989636


In [ ]:
import pandas as pd
import numpy as np
from numba import jit

@jit(nopython=True)
def mape_numba(actual_rating, rating):
    return np.abs(actual_rating - rating) / rating

def MAPE(df: pd.DataFrame) -> float:
    # Удаляем строки с нулевым рейтингом
    df = df[df['rating'] != 0]
    
    # Группируем данные по рецепту
    grouped_data = df.groupby('recipe_id')
    
    # Вычисляем средний рейтинг для каждого рецепта
    mean_ratings = grouped_data['rating'].mean()
    
    # Вычисляем сумму абсолютных процентных отклонений для каждого рецепта
    mape_sum = 0
    for recipe_id, rating in mean_ratings.iteritems():
        recipe_data = df[df['recipe_id'] == recipe_id]
        actual_rating = recipe_data['rating'].values
        mape_sum += mape_numba(actual_rating, rating)
    
    # Вычисляем среднее абсолютное процентное отклонение для всех рецептов
    result = mape_sum.sum() / len(mean_ratings)
    
    return result

#### [версия 2]
* Уточнены формулировки задач 1, 3, 4